<a href="https://colab.research.google.com/github/rebellovcr/Shark-Hunter/blob/main/Shark-Hunter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests
import zipfile
import datetime
from tqdm import tqdm  # Import tqdm for progress bar

def get_date_input():
    while True:
        input_date = input("Digite a data no formato DD-MM-AAAA: ")
        try:
            datetime.datetime.strptime(input_date, "%d-%m-%Y")
            return input_date
        except ValueError:
            print("Formato de data inválido. Digite novamente.")

def download_file_with_progress(url, save_path):
    with requests.get(url, stream=True) as r:
        total_size_in_bytes = int(r.headers.get('content-length', 0))
        block_size = 1024  # 1 Kibibyte
        progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
        with open(save_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=block_size):
                progress_bar.update(len(chunk))
                f.write(chunk)
        progress_bar.close()

def extract_zip_with_progress(zip_file, extract_dir):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        for file in tqdm(file_list, desc='Extracting', unit='files'):
            zip_ref.extract(file, extract_dir)

def filter_and_rewrite_file(input_file):
    temp_file = input_file + ".tmp"
    total_lines = sum(1 for line in open(input_file, 'r'))  # Count total lines in the original file

    with open(input_file, 'r') as infile, open(temp_file, 'w') as outfile:
        progress_bar = tqdm(total=total_lines, desc='Filtrando', unit=' linhas')

        for line in infile:
            columns = line.strip().split(';')
            if len(columns) >= 5:
                try:
                    if float(columns[4].replace(',', '.')) >= 100000:
                        outfile.write(line)
                except ValueError:
                    pass  # Caso não seja um número válido ou a quinta coluna não exista ou seja inválida, não escreva

            progress_bar.update(1)  # Update progress bar for each line processed

        progress_bar.close()

    os.remove(input_file)
    os.rename(temp_file, input_file)

def main():
    input_date = get_date_input()
    split_date = input_date.split('-')
    url_date_format = f"{split_date[2]}-{split_date[1]}-{split_date[0]}"

    url = f"https://arquivos.b3.com.br/apinegocios/tickercsv/{url_date_format}.zip"
    current_directory = os.getcwd()
    zip_file_path = os.path.join(current_directory, f"{url_date_format}.zip")

    print("Baixando arquivo ZIP...")
    download_file_with_progress(url, zip_file_path)
    print(f"Arquivo ZIP baixado com sucesso: {os.path.basename(zip_file_path)}")

    print("Extraindo arquivo ZIP...")
    extract_zip_with_progress(zip_file_path, current_directory)
    print(f"Arquivo ZIP extraído para: {current_directory}")

    os.remove(zip_file_path)
    print(f"Arquivo ZIP deletado: {os.path.basename(zip_file_path)}")

    input_file = f"{input_date}_NEGOCIOSAVISTA.txt"
    filter_and_rewrite_file(input_file)

    print(f"Filtro aplicado e arquivo {input_file} atualizado com sucesso.")

if __name__ == "__main__":
    main()
